# Requests, Beuatifulsoup 핵심 3가지
1. [url -> html]: resp = requests.get(url, params=none)
2. [html -> lxml]: soup = Beautifulsoup(resp.content,'lxml')
3. [lxml query]: tag = soup.select('tag1.class tag2')   [ .text ]  [ .strip() ]
- 원하는 태그를 불러온 후, .text나 .strip()을 적용 가능함

In [2]:
# 필요 패키지
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm.notebook import tqdm

In [3]:
url = 'https://www.naver.com/'
# step 1: url -> HTML
resp = requests.get(url) # HTML
# step 2: HTML -> BeautifulSoup 형변환
soup = BeautifulSoup(resp.content, 'lxml') # XML

In [ ]:
# step 3: 원하는 태그 선택; soup.select(): 원하는 태그를 받아 "리스트"로 반환해주는 함수!!
soup.select('원하는 태그') # tag-element list

참조: css selector
- css selector reference: https://www.w3schools.com/cssref/css_selectors.php
- css tester 연습사이트: https://www.w3schools.com/cssref/trysel.php

In [ ]:
# step 4: 태그에서 정보 파싱
wanted_tag = soup.select('원하는 태그') # 원하는 tag-element가 리스트로 있는 형태
wanted_tag[0].text 
# Beautifulsoup의 .text와 .string은 한 줄의 tag-element만을 입력받으므로
# soup.select로 선택된 리스트에서 텍스트를 뽑아내려면 [0]과 같이 인덱싱을 통해 특정 행을 선택해주어야 함
# 여러 개의 tag-element가 선택되었을 경우 [0] 인덱싱을 통해 구조를 파악하고,
# for문, list comprehension을 통해 전체 행의 문자를 불러올 수 있다

예제 1. soup.select 이용 parsing

In [7]:
#예제
url = "https://search.naver.com/search.naver?where=news&sm=tab_jum&query=날씨"
# step 1: url -> HTML
resp = requests.get(url) # HTML
# step 2: HTML -> BeautifulSoup 형변환
soup = BeautifulSoup(resp.content, 'lxml') # XML
title_tag = soup.select('a.news_tit')
title_tag[0].text

'오송 참사날 "날씨 완벽했다"…싸이 \'여수 흠뻑쇼\' 후기 논란'

In [11]:
# for loop 버전
title = []
for i in range(len(title_tag)):
    title.append(title_tag[i].text)
title

# list comprehension 버전
title = [title_tag[i].text for i in range(len(title_tag))]
title[0:3]

['오송 참사날 "날씨 완벽했다"…싸이 \'여수 흠뻑쇼\' 후기 논란',
 '[내일날씨] 19일도 강한비… 충청·남부 최대 200㎜',
 '오송 참사에 "날씨 완벽했다"…\'흠뻑쇼\' 싸이 또 구설']

예제 1-1. 여러 변수는 dictionary, df 활용

In [3]:
# 여러 번수 가져오고 싶다면? dictionary 활용
url = 'https://www.scrapethissite.com/pages/forms/?page_num=1&per_page=25'
# team, win 변수 가져오기
dic_hk = {'teams' : [], 'Wins' : [], }

resp = requests.get(url)
soup = BeautifulSoup(resp.content, 'lxml')
soup.select('tr.team')[0] # td.name, td.wins를 가져오면 된다


<tr class="team">
<td class="name">
                            Boston Bruins
                        </td>
<td class="year">
                            1990
                        </td>
<td class="wins">
                            44
                        </td>
<td class="losses">
                            24
                        </td>
<td class="ot-losses">
</td>
<td class="pct text-success">
                            0.55
                        </td>
<td class="gf">
                            299
                        </td>
<td class="ga">
                            264
                        </td>
<td class="diff text-success">
                            35
                        </td>
</tr>

In [4]:
for row in soup.select('tr.team'):
    dic_hk['teams'].append(row.select('td.name')[0].text.strip())
    dic_hk['Wins'].append(row.select('td.wins')[0].text.strip())

In [11]:
dic_hk['teams'][0:3], dic_hk['Wins'][0:3]

(['Boston Bruins', 'Buffalo Sabres', 'Calgary Flames'], ['44', '31', '46'])

In [12]:
pd.DataFrame(dic_hk).head()

,teams,Wins
0,Boston Bruins,44
1,Buffalo Sabres,31
2,Calgary Flames,46
3,Chicago Blackhawks,49
4,Detroit Red Wings,34


예제 2. img 긁어오기

In [17]:
url = 'https://davelee-fun.github.io'
req = requests.get(url)
soup = BeautifulSoup(req.content, 'lxml')
img_tag = soup.select('img.featured-box-img-cover')
img_tag[0], img_tag[0]['src'] # img파일 주소

(<img class="featured-box-img-cover" src="https://static.coupangcdn.com/image/retail/images/2018/09/18/11/8/6964dd11-7ff2-448d-8462-db07e3ca2a5f.jpg"/>,
 'https://static.coupangcdn.com/image/retail/images/2018/09/18/11/8/6964dd11-7ff2-448d-8462-db07e3ca2a5f.jpg')

In [18]:
# 방법 1. for loop
src_list = []
for s in soup.select('img.featured-box-img-cover'): 
# 여기서 s는 soup.select('img.featured-box-img-cover')[s] 임
    src_list.append(s['src']) # soup.select('img.featured-box-img-cover')[s]['src'] : 
src_list
# tag 사이의 content를 .text로 불러올 때와는 다르게 tag 내의 src 속성의 value를 가져와야 하므로
# 원하는 attribute(여기서는 'src')로 한번 더 인덱싱해준다

# 방법 2. list comprehension
src_list = [s['src'] for s in soup.select('img.featured-box-img-cover')]

In [ ]:
# 방법 1. for문으로 처리하기
for idx, s in enumerate(src_list):
    img_resp = requests.get(s)
    file_name = f'./mattress/img_{idx}.jpg'
    with open(file_name,'wb') as f: # write binary모드로 열기
        f.write(img_resp.content) # src 내의 content 집어넣기

file r/w

In [ ]:
# open('파일경로','모드')
# 모드를 'w'로 하면 파일이 새로 생성된다
f = open('a.txt','w')

In [ ]:
# with문을 쓰는 이유: with문이 끝나면 file이 자동으로 닫힘
with open('b.txt','w') as f:
    for i in range(10):
        t = f'{i}번째\n'
        f.write(t)